In [1]:
import sys,os
pardir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(pardir)

In [2]:
from scripts.get_data.get_nba_api import retrieve_data
LOAD_DATA_FOR_ALL_DATES = True
LOAD_SPECIFIC_ENDPOINT = [] # e.g. ['boxscoretraditionalv2'], if empty list then all endpoints will be loaded

retrieve_data(all_dates=LOAD_DATA_FOR_ALL_DATES, test=True, specific_endpoints=LOAD_SPECIFIC_ENDPOINT)

No existing CSV file found for players endpoint. A new CSV file will be created.


TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [5]:
def factorial(n, stop_n=0):
    if n <= stop_n:
        return 1
    return n * factorial(n - 1, stop_n=stop_n)
    

START = 4
STOP = 2
draft_history = {
    "endpoint": factorial,
    "parameters": {"n": None, "stop_n": STOP},
    "loop_parameter": "n",
}

params = draft_history["parameters"]
for i in range(START, 0, -1):
    params["n"] = i
    print(params)
    print(draft_history["endpoint"](**params))

{'n': 4, 'stop_n': 2}
12
{'n': 3, 'stop_n': 2}
3
{'n': 2, 'stop_n': 2}
1
{'n': 1, 'stop_n': 2}
1


In [ ]:
dict1 = {
    "a": [1, 2, 3],
    "b": "m",
    "c": ["x", "y"],
}

In [13]:
from itertools import product

dict1 = {
    "a": [1, 2, 3],
    "b": "m",
    "c": ["x", "y"],
}

keys = dict1.keys()
values = dict1.values()

combinations = [dict(zip(keys, combination)) for combination in product(*values)]

print(combinations)

[{'b': 'm', 'c': 'x'}, {'b': 'm', 'c': 'y'}, {'b': 'e', 'c': 'x'}, {'b': 'e', 'c': 'y'}]


In [1]:

from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamelog, scoreboardv2, boxscoretraditionalv2

FIRST_SEASON = 1946  # team_df['year_founded'].min()
LAST_SEASON = 2022

seasons_dict = {
    "name": "seasons",
    "first_season": FIRST_SEASON,
    "last_season": LAST_SEASON,
}

teams_dict = {
    "name": "teams",
    "endpoint": lambda kwargs: teams.get_teams(),
    "main_loop_parameter": None,
    "parent": None,
    "parent_col": None,
    "csv": "teams_data.csv",
}

league_game_log_dict = {
    "name": "league_game_log",
    "endpoint": lambda kwargs: leaguegamelog.LeagueGameLog(**kwargs).league_game_log,
    "main_loop_parameter": "season",
    "parent": teams_dict,
    "parent_col": "year_founded",
    "main_col_dtype": "int",
    "additional_parameters": {
        "season_type_all_star": ["Regular Season", "Pre Season", "Playoffs", "All-Star"]
    },
    "csv": "game_log_data.csv",
}

scoreboard_dict = {
    "name": "scoreboard",
    "endpoint": lambda kwargs: scoreboardv2.ScoreboardV2(**kwargs).game_header,
    "main_loop_parameter": "game_date",
    "parent": league_game_log_dict,
    "parent_col": "GAME_DATE_EST",
    "current_col": "GAME_DATE",
    "main_col_dtype": "datetime64[ns]",
    "csv": "scoreboard_data.csv",
}

box_score_traditional_dict = {
    "name": "box_score_traditional",
    "endpoint": lambda kwargs: boxscoretraditionalv2.BoxScoreTraditionalV2(
        **kwargs
    ).player_stats,
    "main_loop_parameter": "game_id",
    "parent": scoreboard_dict,
    "parent_col": "GAME_ID",
    "main_col_dtype": "int",
    "csv": "box_score_traditional_data.csv",
}